In [1]:
# region General Imports
import os
import re
import uuid
import shutil
import time
import random
import datetime
import glob
import pickle
import tqdm
import copy
import optuna
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import mne
from rich import print as rprint
from rich.pretty import pprint as rpprint
from tqdm import tqdm
from itertools import chain
from functools import partial
# endregion General Imports

import tempfile
import tensorflow as tf
import numpy as np
from tensorflow import keras

import tensorflow_model_optimization as tfmot
%load_ext tensorboard
    
# os.environ["KERAS_BACKEND"] = "tf"
# os.environ["TF_USE_LEGACY_KERAS"] = "0"
# import jax
# import jax.numpy as jnp
# import keras
    
# region Keras
from keras.models import Model
from keras.layers import Dense, Activation, Permute, Dropout
from keras.layers import (
    Conv2D,
    MaxPooling2D,
    AveragePooling2D,
    Conv1D,
    MaxPooling1D,
    AveragePooling1D,
)
from keras.layers import SeparableConv2D, DepthwiseConv2D
from keras.layers import BatchNormalization
from keras.layers import SpatialDropout2D
from keras.regularizers import l1_l2
from keras.layers import Input, Flatten
from keras.constraints import max_norm
from keras import backend as K
# endregion Keras

from custom_datasets.fatigue_mi import FatigueMI

# Sklearn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, TimeDistributed, BatchNormalization
from sklearn import preprocessing

from model_optim.utils import channels_to_channels_idx

/home/arazzz/anaconda3/envs/moabb_model_optimization_quant/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-24 17:47:06.576219: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 17:47:06.576317: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 17:47:06.577275: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 17:47:06.586255: I tensorflow/core/platform/cpu_feature_guard

To use the get_shape_from_baseconcar, InputShapeSetterEEG, BraindecodeDatasetLoaderyou need to install `braindecode`.`pip install braindecode` or Please refer to `https://braindecode.org`.


/home/arazzz/anaconda3/envs/moabb_model_optimization_quant/lib/python3.11/site-packages/moabb/pipelines/__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(


In [2]:
SKLRNG = 42
# RNG = jax.random.PRNGKey(SKLRNG)

In [3]:
# region Helper funcs
def shallow_conv_net_square_layer(x):
    return tf.math.square(x)

def shallow_conv_net_log_layer(x):
    return tf.math.log(tf.clip_by_value(x, 1e-7, 10000))

CUSTOM_OBJECTS = {
    "shallow_conv_net_square_layer": shallow_conv_net_square_layer, 
    "shallow_conv_net_log_layer": shallow_conv_net_log_layer 
}
# endregion Helper funcs

# region Models
def shallow_conv_net(
    nb_classes, channels, samples, **kwargs
):
    """
    From: https://github.com/vlawhern/arl-eegmodels/blob/master/EEGModels.py
    """

    _POOL_SIZE_D2_ = kwargs.get("pool_size_d2", 35)
    _STRIDES_D2_ = kwargs.get("strides_d2", 7)
    _CONV_FILTERS_D2_ = kwargs.get("conv_filters_d2", 13)

    _POOL_SIZE_ = kwargs.get("pool_size", (1, _POOL_SIZE_D2_))
    _STRIDES_ = kwargs.get("strides", (1, _STRIDES_D2_))
    _CONV_FILTERS_ = kwargs.get("conv_filters", (1, _CONV_FILTERS_D2_))

    _CONV2D_1_UNITS_ = kwargs.get("conv2d_1_units", 40)
    _CONV2D_2_UNITS_ = kwargs.get("conv2d_2_units", 40)
    _L2_REG_1_ = kwargs.get("l2_reg_1", 0.01)
    _L2_REG_2_ = kwargs.get("l2_reg_2", 0.01)
    _L2_REG_3_ = kwargs.get("l2_reg_3", 0.01)
    _DROPOUT_RATE_ = kwargs.get("dropout_rate", 0.5)

    input_main = Input(shape=(channels, samples, 1))
    block1 = Conv2D(
        _CONV2D_1_UNITS_,
        _CONV_FILTERS_,
        input_shape=(channels, samples, 1),
        kernel_constraint=max_norm(2.0, axis=(0, 1, 2)),
        kernel_regularizer=keras.regularizers.L2(_L2_REG_1_),
    )(input_main)
    # block1       = Conv2D(40, (channels, 1), use_bias=False,
    #                       kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block1 = Conv2D(
        _CONV2D_2_UNITS_,
        (channels, 1),
        use_bias=False,
        kernel_constraint=max_norm(2.0, axis=(0, 1, 2)),
        kernel_regularizer=keras.regularizers.L2(_L2_REG_2_),
    )(block1)
    block1 = BatchNormalization(epsilon=1e-05, momentum=0.9)(block1)
    block1 = Activation(shallow_conv_net_square_layer)(block1)
    block1 = AveragePooling2D(pool_size=_POOL_SIZE_, strides=_STRIDES_)(block1)
    block1 = Activation(shallow_conv_net_log_layer)(block1)
    block1 = Dropout(_DROPOUT_RATE_)(block1)
    flatten = Flatten()(block1)
    # dense        = Dense(nb_classes, kernel_constraint = max_norm(0.5))(flatten)
    dense = Dense(
        nb_classes,
        kernel_constraint=max_norm(0.5),
        kernel_regularizer=keras.regularizers.L2(_L2_REG_3_),
    )(flatten)
    softmax = Activation("softmax")(dense)

    return Model(inputs=input_main, outputs=softmax)

# endregion Models

In [4]:
subject_best_trials = glob.glob('./temp_v2/**/model/study_best_trial.npy', recursive=True)
subject_best_trials = sorted(subject_best_trials, key=lambda x: os.path.getmtime(x))
subject_best_trials = sorted(subject_best_trials, key=lambda x: int(re.compile(r"\[.*\]").search(x).group(0).strip("[]")))

In [5]:
def data_generator(dataset, subjects = [1], channel_idx = [], filters = ([8, 32],), sfreq = 250):

    find_events = lambda raw, event_id: mne.find_events(raw, shortest_event=0, verbose=False) if len(mne.utils._get_stim_channel(None, raw.info, raise_error=False)) > 0 else mne.events_from_annotations(raw, event_id=event_id, verbose=False)[0]
    
    data = dataset.get_data(subjects=subjects)
    
    X = []
    y = []
    metadata = []

    for subject_id in data.keys():
        for session_id in data[subject_id].keys():
            for run_id in data[subject_id][session_id].keys():
                raw = data[subject_id][session_id][run_id]
                
                for fmin, fmax in filters:
                    raw = raw.filter(l_freq = fmin, h_freq = fmax, method = 'iir', picks = 'eeg', verbose = False)
                
                events = find_events(raw, dataset.event_id)

                tmin = dataset.interval[0]
                tmax = dataset.interval[1]

                channels = np.asarray(raw.info['ch_names'])[channel_idx] if len(channel_idx) > 0 else np.asarray(raw.info['ch_names'])

                # rpprint(channels)
                
                stim_channels = mne.utils._get_stim_channel(None, raw.info, raise_error=False)
                picks = mne.pick_channels(raw.info["ch_names"], include=channels, exclude=stim_channels, ordered=True)

                x = mne.Epochs(
                    raw,
                    events,
                    event_id=dataset.event_id,
                    tmin=tmin,
                    tmax=tmax,
                    proj=False,
                    baseline=None,
                    preload=True,
                    verbose=False,
                    picks=picks,
                    event_repeated="drop",
                    on_missing="ignore",
                )
                x_events = x.events
                inv_events = {k: v for v, k in dataset.event_id.items()}
                labels = [inv_events[e] for e in x_events[:, -1]]

                # rpprint({
                #     "X": np.asarray(x.get_data(copy=False)).shape,
                #     "y": np.asarray(labels).shape,
                #     "channels selected": np.asarray(raw.info['ch_names'])[channel_idx]
                # })

                # x.plot(scalings="auto")
                # display(x.info)
                
                x_resampled = x.resample(sfreq) # Resampler_Epoch
                x_resampled_data = x_resampled.get_data(copy=False) # Convert_Epoch_Array
                x_resampled_data_standard_scaler = np.asarray([
                    StandardScaler().fit_transform(x_resampled_data[i])
                    for i in np.arange(x_resampled_data.shape[0])
                ]) # Standard_Scaler_Epoch

                # x_resampled.plot(scalings="auto")
                # display(x_resampled.info)

                n = x_resampled_data_standard_scaler.shape[0]
                # n = x.get_data(copy=False).shape[0]
                met = pd.DataFrame(index=range(n))
                met["subject"] = subject_id
                met["session"] = session_id
                met["run"] = run_id
                x.metadata = met.copy()
                
                # X.append(x_resampled_data_standard_scaler)
                X.append(x)
                y.append(labels)
                metadata.append(met)

    return np.concatenate(X, axis=0), np.concatenate(y), pd.concat(metadata, ignore_index=True)

fat_dataset = FatigueMI()

In [8]:
rpprint(subject_best_trials)

[
│   './temp_v2/[1]/292cbc92b8cf46da9986fe7d8447819f/model/study_best_trial.npy',
│   './temp_v2/[2]/6405d11e654b42aca9df48458c67ecde/model/study_best_trial.npy',
│   './temp_v2/[3]/9c1b753483db409a90eab7b7149b8af8/model/study_best_trial.npy',
│   './temp_v2/[4]/3623cb4ba1ad4a908c9098f5297a6778/model/study_best_trial.npy',
│   './temp_v2/[5]/d199c9c2ac924b238693f158eb88f675/model/study_best_trial.npy',
│   './temp_v2/[6]/39d01251ff494106bf04f8a2cffcdd74/model/study_best_trial.npy',
│   './temp_v2/[7]/aabe056cd1954a6f92ab47d84c86b1b8/model/study_best_trial.npy',
│   './temp_v2/[8]/9fd82ec44ef3496da6307b57ecf4532f/model/study_best_trial.npy',
│   './temp_v2/[9]/63a8c87ffc02471893db5ac9a0781946/model/study_best_trial.npy',
│   './temp_v2/[10]/c2cc69dca74d4bfa81722cd634e6403e/model/study_best_trial.npy',
│   './temp_v2/[11]/e0643f9a780146a4adc15ddd4a9ff053/model/study_best_trial.npy',
│   './temp_v2/[12]/96dc576945fb4f2db582d66ae1d2c8ce/model/study_best_trial.npy',
│   './temp_v2/[13]/1b189965ada44ff99e73fa145cd3901d/model/study_best_trial.npy',
│   './temp_v2/[14]/a3304348c7094d02a024828ede942cda/model/study_best_trial.npy'
]

In [9]:
for subject_best_trial in subject_best_trials[0:1]:
    
    rprint("\n\n\nLoading best trial model...", subject_best_trial)

    model = np.load(subject_best_trials[0], allow_pickle=True).item()
    model_info = {
        "subject": model.user_attrs["trial_data"]["subject"] if hasattr(model.user_attrs["trial_data"], "subject") else int(re.compile(r"\[.*\]").search(model.user_attrs["trial_data"]["data_path"]).group(0).strip("[]")),
        "sfreq": model.params["sfreq"] if "sfreq" in model.params else 128,
        "batch_size": model.params["batch_size"] if "batch_size" in model.params else 128,
        "channels_selected": model.user_attrs["trial_data"]["channels_selected"],
        "channels_idx_selected": channels_to_channels_idx(model.user_attrs["trial_data"]["channels_selected"], fat_dataset.get_data(subjects=[1])[1]['0']['0'].info['ch_names'][:-1]),
        "model": tf.keras.models.model_from_json(model.user_attrs["trial_data"]["model"], custom_objects=CUSTOM_OBJECTS),
        "test_acc": model.user_attrs["trial_data"]["test_accuracy"]
    }
    if "weights" in model.user_attrs["trial_data"]:
        model_info["model"].set_weights(model.user_attrs["trial_data"]["weights"])
    elif "model_weights" in model.user_attrs["trial_data"]:
        model_info["model"].set_weights(model.user_attrs["trial_data"]["model_weights"])


    # region 1) ------------- Build & save baseline model ------------- 
    rprint("Building & saving baseline model...")
    X, y, _ = data_generator(fat_dataset, subjects=[model_info["subject"]], channel_idx=model_info["channels_idx_selected"], sfreq=model_info["sfreq"])
    y_encoded = LabelEncoder().fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=SKLRNG, shuffle=True, stratify=y_encoded)

    model_info["model"].compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    baseline_model_test = model_info["model"].evaluate(X_test, y_test, batch_size=model_info["batch_size"])

    baseline_test_acc = baseline_model_test[1]

    _, keras_file = None, subject_best_trial.split("study_best_trial.npy")[0] + "baseline_model.h5"
    tf.keras.models.save_model(model_info["model"], keras_file, include_optimizer=False)
    rprint('Saved baseline model to:', keras_file)
    # endregion

    # region 2) ------------- Pruning the baseline model -------------
    rprint("Pruning the baseline model...")
    prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
    batch_size = model_info["batch_size"]
    epochs = 2
    end_step = np.ceil(len(X_train) / batch_size).astype(np.int32) * epochs
    # end_step = model_info["batch_size"]
    # Define model for pruning.
    pruning_params = {
        #   'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=0.9,
        #                                                           begin_step=0,
        #                                                           frequency=100)
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                                final_sparsity=0.90,
                                                                begin_step=0,
                                                                end_step=end_step)
    }
    keras.utils.get_custom_objects().update({
        **CUSTOM_OBJECTS
    })
    baseline_model_copy = tf.keras.models.clone_model(model_info["model"])
    model_for_pruning = prune_low_magnitude(baseline_model_copy, **pruning_params)
    # `prune_low_magnitude` requires a recompile.
    model_for_pruning.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    logdir = "logs/pruning/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    # tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

    callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
    ]

    model_for_pruning.fit(X_train, y_train, batch_size=model_info["batch_size"], epochs=epochs, validation_split=0.2, callbacks=callbacks)
    _, model_for_pruning_accuracy = model_for_pruning.evaluate(X_test, y_test, verbose=0)
    rpprint({
        'Baseline test accuracy': baseline_test_acc,
        'Pruned test accuracy': model_for_pruning_accuracy
    })

    model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
    _, pruned_keras_file = None, subject_best_trial.split("study_best_trial.npy")[0] + "pruned_model.h5"
    tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
    rprint('Saved pruned Keras model to:', pruned_keras_file)

    # endregion

    # region 3) ------------- Converting pruned model to TFLite -------------
    converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
    pruned_tflite_model = converter.convert()

    _, pruned_tflite_file = None, subject_best_trial.split("study_best_trial.npy")[0] + "pruned_model.tflite"

    with open(pruned_tflite_file, 'wb') as f:
        f.write(pruned_tflite_model)

    rprint('Saved pruned TFLite model to:', pruned_tflite_file)

    def get_gzipped_model_size(file):
        # Returns size of gzipped model, in bytes.
        import os
        import zipfile
        _, zipped_file = tempfile.mkstemp('.zip')
        with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
            f.write(file)
        
        return os.path.getsize(zipped_file)

    rprint("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
    rprint("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
    rprint("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

    # endregion

    # region 4) ------------- Quantizing pruned TFLite model ------------- 
    converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    quantized_and_pruned_tflite_model = converter.convert()

    _, quantized_and_pruned_tflite_file = None, subject_best_trial.split("study_best_trial.npy")[0] + "quantized_and_pruned_model.tflite"

    with open(quantized_and_pruned_tflite_file, 'wb') as f:
        f.write(quantized_and_pruned_tflite_model)

    rprint('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)
    rprint("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
    rprint("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))
    # endregion

    # region 5) ------------- Evaluation check ------------- 
    def evaluate_model(interpreter):
        input_index = interpreter.get_input_details()[0]["index"]
        output_index = interpreter.get_output_details()[0]["index"]

        # signatures = interpreter.get_signature_list()
        # rprint(interpreter.get_input_details(), interpreter.get_output_details(), signatures)

        # Run predictions on every image in the "test" dataset.
        predictions = []
        for i, v in enumerate(X_test):
            v = v[np.newaxis, :, :, np.newaxis].astype(np.float32)
            # if i % 1000 == 0:
            #   rprint('Evaluated on {n} results so far.'.format(n=i))
            # # Pre-processing: add batch dimension and convert to float32 to match with
            # # the model's input data format.
            # v = np.expand_dims(v, axis=0).astype(np.float32)
            interpreter.set_tensor(input_index, v)

            # Run inference.
            interpreter.invoke()

            # Post-processing: remove batch dimension and find the digit with highest
            # probability.
            output = interpreter.tensor(output_index)
            class_prediction = np.argmax(output()[0]) # 0 = left, 1 = right
            predictions.append(class_prediction)

        print('\n')
        # Compare prediction results with ground truth labels to calculate accuracy.
        predictions = np.asarray(predictions)
        accuracy = (predictions == y_test).mean()
        return accuracy

    interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
    interpreter.allocate_tensors()

    test_accuracy = evaluate_model(interpreter)

    rprint('Pruned and quantized TFLite test_accuracy:', test_accuracy)
    rprint('Pruned TF test accuracy:', model_for_pruning_accuracy)
    # endregion


Loading best trial model... ./temp_v2/[2]/6405d11e654b42aca9df48458c67ecde/model/study_best_trial.npy

Building & saving baseline model...

Sampling frequency of the instance is already 300.0, returning unmodified.
Adding metadata with 3 columns
1/1 [==============================] - 1s 836ms/step - loss: 25.5757 - accuracy: 0.8636


/tmp/ipykernel_38612/1323652598.py:34: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model_info["model"], keras_file, include_optimizer=False)


Saved baseline model to: ./temp_v2/[2]/6405d11e654b42aca9df48458c67ecde/model/baseline_model.h5

...

Saved quantized and pruned TFLite model to: 
./temp_v2/[2]/6405d11e654b42aca9df48458c67ecde/model/quantized_and_pruned_model.tflite

Size of gzipped baseline Keras model: 240971.00 bytes

Size of gzipped pruned and quantized TFlite model: 66992.00 bytes

Pruned and quantized TFLite test_accuracy: 0.45454545454545453

Pruned TF test accuracy: 0.4545454680919647